# Return a list of names from Wikipedia

The input for 'Return_death_dates' is a list of names. I need to query the Wikipedia API and return a list of names. Ideally this should be a list of all people on wikipedia. Extra desirable information is profession (i.e., muscician), music genre, success.

## Use the Wikipedia API

In [1]:
import wikipediaapi

wiki_en = wikipediaapi.Wikipedia('en')  # call an instance of english

## Explore .categories

In [18]:
cat_Musicians_page = wiki_en.page("Category:Musicians")

f'Using the .page() method on a Category returns a {type(cat_Musicians_page)}.'

"Using the .page() method on a Category returns a <class 'wikipediaapi.WikipediaPage'>."

In [19]:
cat_Musicians_members = cat_Musicians_page.categorymembers

f'We can then use the attribute .categorymembers to return a {type(cat_Musicians_members)}'

"We can then use the attribute .categorymembers to return a <class 'dict'>"

In [21]:
length = len(list(cat_Musicians_members.keys())) # number of keys in dict

import random

onesample = random.choice(list(cat_Musicians_members.items()))

f'The dictionary has {length} elements of the following form: {onesample}'

"The dictionary has 65 elements of the following form: ('Category:Singers', Category:Singers (id: ??, ns: 14))"

Of note is the tuple that includes 'ns'. This tells us that there are 14 more levels that we can dig.

In [79]:
def wiki_cat_dictionary(wiki_Category):
    """
    Using a category input, return a dictionary.
    
    category syntax == "Category:Musicians"
    """
    wiki_en = wikipediaapi.Wikipedia('en')
    
    return wiki_en.page(wiki_Category).categorymembers

In [82]:
wiki_cat_dictionary("Category:Musicians")

{'Musician': Musician (id: ??, ns: 0),
 'Songster': Songster (id: ??, ns: 0),
 'Troubadour': Troubadour (id: ??, ns: 0),
 'Virtuoso': Virtuoso (id: ??, ns: 0),
 'Category:Musicians by band': Category:Musicians by band (id: ??, ns: 14),
 'Category:Musicians by century': Category:Musicians by century (id: ??, ns: 14),
 'Category:Musicians by genre': Category:Musicians by genre (id: ??, ns: 14),
 'Category:Musicians by geographical categorization': Category:Musicians by geographical categorization (id: ??, ns: 14),
 'Category:Musicians by instrument': Category:Musicians by instrument (id: ??, ns: 14),
 'Category:Artists by record label': Category:Artists by record label (id: ??, ns: 14),
 'Category:Musicians of Indian descent': Category:Musicians of Indian descent (id: ??, ns: 14),
 'Category:Musicians of Iranian descent': Category:Musicians of Iranian descent (id: ??, ns: 14),
 'Category:Lists of musicians': Category:Lists of musicians (id: ??, ns: 14),
 'Category:Musical groups': Catego

In [92]:
def list_bottomcategorymembers(categorymembers, list_name):
    """
    Using a dictionary of wikipedia page category members, return a list of all bottom category members
    """
    
    bottom_categories = []
    
    for c in categorymembers.values():
        if c.ns == 0:
            bottom_categories.append((c.title, c.ns))
            print(len(bottom_categories))
        elif c.ns == wikipediaapi.Namespace.CATEGORY:
            list_bottomcategorymembers(c.categorymembers)
            
    return bottom_categories

# Issue is that this code keeps wiping the bottom_categories list

In [93]:
test_list = list_bottomcategorymembers(wiki_cat_dictionary("Category:Musicians"))

1
2
3
4
1
2
1
2
3
4
1
2
3
1
2
3
4
5
6
7
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
1
2
3
4
5
6
7
1
2
3
1
2
1
2
3
4
1
2
3
4
5
1
2
3
4
5
1
2
3
1
2
3
1
2
1
2
3
4
1
2
3
4
5
6
7
1
2
3
4
1
2
3
4
5
6
7
1
2
3
4
5
6
7
1
2
1
2
3
4
1
2
3
4
5
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
1
2
3
4
5
1
2
3
4
1
2
1
2
3
4
1
2
3
4
5
6
7
8
9
1
2
3
4
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
1
2
3
4
1
2
3


KeyboardInterrupt: 

In [72]:
# code source: https://pypi.org/project/Wikipedia-API/

def print_categorymembers(categorymembers, max_items, level=0, max_level=1):
    item_num = 0
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, max_items=max_items, level=level + 1, max_level=max_level)
        
        item_num = item_num + 1
        if max_items == item_num:
            break

In [73]:
print_categorymembers(wiki_cat_dictionary('Category:Musicians by genre'), max_items = 3, max_level=14)

*: Revivalist artist (ns: 0)
*: Category:Musical groups by genre (ns: 14)
**: Category:Musical groups by genre and nationality (ns: 14)
***: Category:Funk musical groups by nationality (ns: 14)
****: Category:American funk musical groups (ns: 14)
*****: 24-Carat Black (ns: 0)
*****: 94 East (ns: 0)
*****: 100 Monkeys (ns: 0)
****: Category:Australian funk musical groups (ns: 14)
*****: The Bamboos (funk band) (ns: 0)
*****: Cookin' on 3 Burners (ns: 0)
*****: Directions In Groove (ns: 0)
****: Category:Austrian funk musical groups (ns: 14)
*****: Supermax (band) (ns: 0)
***: Category:Blues musical groups by nationality (ns: 14)
****: Category:American blues musical groups (ns: 14)
*****: 7th Order (ns: 0)
*****: A.B. Skhy (ns: 0)
*****: Andy Frasco & The U.N. (ns: 0)
****: Category:Australian blues musical groups (ns: 14)
*****: Blue Ruin (band) (ns: 0)
*****: Bondi Cigars (ns: 0)
*****: Carson (band) (ns: 0)
****: Category:Bangladeshi blues musical groups (ns: 14)
*****: Feedback (ban